In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import os
os.chdir('/content/drive/My Drive/Hyperspectral_Image_Classification/code')

In [2]:
from utils import *
import scipy.io as sio

#  Load Target Dataset - Botswana

In [3]:
uBotswana = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/code/Datasets/Botswana.mat')
gt_Botswana = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/code/Datasets/Botswana_gt.mat')

In [4]:
data = uBotswana['Botswana']
ground_truth = gt_Botswana['Botswana_gt']

In [5]:
data.shape

(1476, 256, 145)

In [6]:
ground_truth.shape

(1476, 256)

# Distribution of Samples in target dataset

In [7]:
class_distribution = pd.DataFrame(np.unique(ground_truth, return_counts = True))
class_distribution = class_distribution.transpose()
class_distribution.columns = ['class','samples']
class_distribution

,class,samples
0,0,374608
1,1,270
2,2,101
3,3,251
4,4,215
5,5,269
6,6,269
7,7,259
8,8,203
9,9,314


In [8]:
classes , counts = np.unique(ground_truth, return_counts = True)
classes = classes[1:] ## Not considering background
classes

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
      dtype=uint8)

# Load saved source model trained on Indian Pines

In [9]:
cm = transfer_learning(source_dataset = 'indian_pines',
                      target_dataset = 'botswana',
                      data = data,
                      ground_truth = ground_truth, 
                      source_training_size = 200,
                      training_samples_from_each_class = 30,
                      test_samples_from_each_class = 10,
                      classes = classes,
                      overlap_ratio = 1,
                      channels = 64,
                      cube_size = 20,
                      learning_rate = 0.0001,
                      epochs = 50,
                      batch_size = 20)

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20, 20, 64)] 0                                            
__________________________________________________________________________________________________
conv_initial (Conv2D)           (None, 10, 10, 64)   36928       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 10, 10, 64)   256         conv_initial[0][0]               
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 10, 10, 64)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [10]:
cm

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,classfication_accuracies
0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,100.0
1,0,9,0,0,1,0,0,0,0,0,0,0,0,0,90.0
2,0,0,5,0,0,0,5,0,0,0,0,0,0,0,50.0
3,0,9,0,1,0,0,0,0,0,0,0,0,0,0,10.0
4,0,1,0,0,9,0,0,0,0,0,0,0,0,0,90.0
5,0,9,0,0,1,0,0,0,0,0,0,0,0,0,0.0
6,7,0,0,0,0,0,3,0,0,0,0,0,0,0,30.0
7,0,0,0,0,0,0,0,10,0,0,0,0,0,0,100.0
8,0,7,0,0,0,0,0,0,3,0,0,0,0,0,30.0
9,0,0,0,0,0,0,0,0,0,9,0,0,1,0,90.0


In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
source_model = load_model('/content/drive/My Drive/Hyperspectral_Image_Classification/code/Trained_models/sub_models/indian_pines _with_200_samples_from_each_class_in_training_set.h5')

In [ ]:
source_model.summary()

In [ ]:
def model_transfer(input_shape, classes):
    X_input = Input(input_shape)

    X = Dense(256, input_dim = X_input.shape, activation='relu', name = 'fc_256',
              kernel_initializer = glorot_uniform(seed = 0))(X_input)
    X = Dense(classes, input_dim = X.shape, activation='softmax', name='fc' + str(classes),
              kernel_initializer = glorot_uniform(seed = 0))(X)
    model = Model(inputs = X_input, outputs = X, name = "model_transfer")
    return model

In [ ]:
X_train, X_test, y_train, y_test, counts_test_set, class_samples = sample_extraction(classes, 
                                                                                    cube_size = 20, 
                                                                                    data = data, 
                                                                                    ground_truth = ground_truth, 
                                                                                    cubes = [], 
                                                                                    output_class = [], 
                                                                                    training_samples_from_each_class = 30,
                                                                                    overlap_ratio = 1, 
                                                                                    channels = 64)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
X_train_transfer = source_model.predict(X_train)
X_test_transfer = source_model.predict(X_test)
print('X_train_transfer => ' + str(X_train_transfer.shape) + '\n' +
      'X_test_transfer  => ' + str(X_test_transfer.shape) + '\n' +
      'y_train => ' + str(y_train.shape) + '\n' +
      'y_test  => ' + str(y_test.shape) + '\n')

In [ ]:
target_model = model_transfer(input_shape = X_train_transfer[0].shape, classes = len(y_train[0]))
target_model.summary()

In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/My Drive/Hyperspectral_Image_Classification/code/Trained_models/transferred_models/fine_tune_on_botswana.h5',
                                    monitor = 'val_categorical_accuracy', 
                                    verbose = 1, 
                                    save_best_only = True)
target_model.compile(optimizer = keras.optimizers.SGD(lr = 0.0001, 
                                                      decay = 1e-5, 
                                                      momentum = 0.9, 
                                                      nesterov = True),
                    loss = 'categorical_crossentropy', 
                    metrics = ['categorical_accuracy'])

target_model.fit(X_train_transfer, y_train, 
                 epochs = 100, 
                 batch_size = 20,
                 validation_data = (X_test_transfer, y_test), 
                 verbose = 1, 
                 callbacks = [model_checkpoint])

preds = target_model.evaluate(X_test_transfer, y_test)
print("Test Accuracy = " + str(preds[1]))
y_pred = target_model.predict(X_test_transfer, verbose = 1)
confusion_matrix = sklearn.metrics.confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))

print(confusion_matrix)
print(counts_test_set)


In [ ]:
import pandas as pd
d = pd.DataFrame(confusion_matrix)

In [ ]:
d.columns = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]

In [ ]:
d.index = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]

In [ ]:
d

In [ ]:
counts_test_set

In [ ]:
d = d.append(pd.DataFrame(counts_test_set.reshape(1,-1), columns = list(d)), ignore_index = True)

In [ ]:
d

In [ ]:
d = d.rename(index={d.index[-1]: 'Total Samples'})

In [ ]:
d

In [ ]:
correct_predictions = np.diag(d)

In [ ]:
correct_predictions


In [ ]:
counts_test_set

In [ ]:
np.divide(correct_predictions/counts_test_set)

In [ ]:
classification_accuracies = np.round((correct_predictions / counts_test_set) * 100, 2)

In [ ]:
d['classfication_accuracies'] = np.round(classification_accuracies,3)

UFuncTypeError: ignored

In [ ]:
classification_accuracies = np.append(classification_accuracies, '-')

In [ ]:
classification_accuracies

array([ 84.17,  98.59,  53.85,  29.73,  33.05,  17.99,  86.46,   1.16,
        24.3 ,  98.17,  79.27, 100.  ,  58.77,  75.38])

In [ ]:
d

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,classfication_accuracies
0,202,14,0,0,14,10,0,0,0,0,0,0,0,0,84.16666666666667
1,0,70,0,0,1,0,0,0,0,0,0,0,0,0,98.59154929577466
2,2,0,119,42,0,0,31,0,5,22,0,0,0,0,53.84615384615385
3,0,0,20,55,0,0,7,3,67,3,0,0,30,0,29.72972972972973
4,16,43,0,5,79,29,0,0,15,0,0,16,36,0,33.054393305439326
5,0,35,0,2,3,43,0,0,12,0,0,45,99,0,17.99163179916318
6,31,0,0,0,0,0,198,0,0,0,0,0,0,0,86.46288209606988
7,0,0,14,0,0,1,5,2,0,5,0,134,12,0,1.1560693641618496
8,0,82,0,63,0,0,0,4,69,0,0,0,66,0,24.295774647887324
9,0,0,0,0,0,0,0,0,0,214,4,0,0,0,98.1651376146789
